In [117]:
import hashlib
import math
import random
import numpy as np
import time

In [118]:
class Statistics:
    n_points = 0
    n_trails = 0
    n_cycles = 0

In [119]:
def trail(fhash,  k , l):
    """
    retourne un triplet (x0, xd, d) 
    fhash: une fonction d'hachage quelconque (md5, sha256)
    k: longueur de la chaine générer
    l: condition d'arret pour le point distingué : les l derniers bits doit etre nul
    """
    x0 = random.getrandbits(k)
    tmp = x0 # c'est un int
    d = 0
    max_it = 20/ (1/(2**l))  # d'apres l'article
    # 2**(l+2)
    mask_k = 2**k - 1
    mask_l = 2**l - 1
    lbyte = math.ceil(l/8)
    while True:
        if d == max_it:
            #print("Risque de cycle")
            return None
        
        if tmp & mask_l == 0:  # condition d'arret
            xd = tmp
            break
        d += 1
        x = tmp.to_bytes(16, byteorder='big')  
        y = fhash(x).digest()
        Statistics.n_points += 1
        tmp = int.from_bytes(y, byteorder='big') & mask_k
        #print(d, tmp)
    
    return (x0, xd, d)

In [ ]:
def collision_detection(fhash, k, l):
    """
    detecte une seule collision
    retourne le couple de triplet ( (x0,xd,d), (x0',xd,d') )
    """
    dico = {}
    while True: 
        res = trail(fhash, k, l)
        Statistics.n_trails += 1
        if res == None:
            Statistics.n_cycles += 1
            continue
            
        x0, xd, d = res
        
        if xd in dico:
            print("Collision found")
            return ( (x0,xd,d),(dico[xd][0], xd,dico[xd][1])  )  
        
        dico[xd] = (x0, d)

In [88]:
k = 50
l = 10
log_nF = k/2 + 0.5*math.log(math.pi/2)
print("On s'attend à 2^{:.1f} évaluations de F".format(log_nF))
print("On s'attend à 2^{:.1f} trails".format(log_nF - l))
debut = time.time()
print(collision_detection(hashlib.md5, 50, 10))
fin = time.time()
print("Temps : {:.1f}s".format(fin - debut))

On s'attend à 2^25.2 évaluations de F
On s'attend à 2^15.2 trails
Collision found
((163275221851853, 552532737869824, 2225), (155129268956935, 552532737869824, 3029))
Temps : 73.7s


In [89]:
print("--> 2^{:.1f} points".format(math.log(Statistics.n_points, 2)))
print("--> 2^{:.1f} trails".format(math.log(Statistics.n_trails, 2)))
print("--> 2^{:.1f} cycles".format(math.log(Statistics.n_cycles, 2)))

--> 2^26.3 points
--> 2^16.4 trails
--> 2^10.6 cycles


In [120]:
def f_cut_k(fhash, val, k):
    """
    Retourne la valeur retournée par fhash(val) tronquée à k bits
    fhash
    val : int
    k
    """
    x = val.to_bytes(16, byteorder='big')  
    y = fhash(x).digest()
    mask_k = 2**k - 1
    return int.from_bytes(y, byteorder='big') & mask_k

In [230]:
def remonter (fhash ,A , B, k, b):
    """
    returne (x,y) tq x != y et fhash(x) == fhash(y)
    A, B : triplet (x0, xd, d)
    b : 0 ou 1
    """       
    
    if A[2] >= B[2]: 

        x = A[0]
        for _ in range(A[2]-B[2]):
            x = f_cut_k(fhash(b), x, k)
        y = B[0]

        if x == y : 
            #print('pb : x==y et fhash(x)==fhash(y)')
            return None

        while True:
            if x == y :
                break

            tmp1 = x            
            tmp2 = y             # anciennes valeurs
            x = f_cut_k(fhash(b), tmp1, k)  
            y = f_cut_k(fhash(1-b), tmp2, k)  

        return ( (tmp1, fhash(b).__name__ ) , (tmp2, fhash(1-b).__name__ ))
        #return (tmp1, tmp2)
        
    else:    # A[2] < B[2] mais on fait la meme chose
        
        y = B[0]
        for _ in range(B[2]-A[2]):
            y = f_cut_k(fhash(1-b), y, k)
        x = A[0]

        if x == y : 
            #print('pb : x==y et fhash(x)==fhash(y)')
            return None

        while True:
            if x == y :
                break

            tmp1 = x            
            tmp2 = y             # anciennes valeurs
            x = f_cut_k(fhash(b), tmp1, k) 
            y = f_cut_k(fhash(1-b), tmp2, k)  

        return ( (tmp1, fhash(b).__name__ ) , (tmp2, fhash(1-b).__name__ ))
        #return (tmp1, tmp2)

In [208]:
def F(b):
    """
    Choisir uniformement une fonction de hash
    b : 0 ou 1
    """
    if b == 0:
        return hashlib.md5
    if b == 1:
        return hashlib.sha256

In [209]:
def collision_detection2(fhash,  k, l):
    """
    detecte une seule collision
    fhash : prend un int en parametre et returne un pointer de fonction
            de hash
    retourne le couple de triplet ( (x0,xd,d), (x0',xd,d') )
    """
    dico = {}
    while True: 
        b = random.randint(0,1) 
        res = trail(fhash(b), k, l)
        Statistics.n_trails += 1
        if res == None:
            Statistics.n_cycles += 1
            continue
            
        x0, xd, d = res
        
        if (xd,1-b) in dico:  
            #print("Collision found")
            
            A = (x0,xd,d)                                  # b
            B = (dico[(xd,1-b)][0], xd,dico[(xd,1-b)][1])  # 1-b
            return remonter(fhash, A , B, k, b )
        dico[(xd,b)] = (x0, d)


In [227]:
def collision_detection_multiple(fhash, k, l, nb_col):
    liste = []
    i = 0
    while i<nb_col :
        tmp = collision_detection2(fhash,  k, l)
        if tmp == None:
            continue
        if tmp in liste or (tmp[1],tmp[0]) in liste: # si collision deja trouvé 
            continue 
        liste.append(tmp)
        i += 1
        #print((liste))
    return liste

In [228]:
# collision_detection_multiple(F, 20, 10, (2**10)/2 )

In [231]:
k = 20
l = 10

collision_detection2(F, k, l)

((222001, 'openssl_md5'), (913729, 'openssl_sha256'))

In [234]:
print(f_cut_k(hashlib.md5, 222001, 20),'\n')
print(f_cut_k(hashlib.sha256, 913729, 20))

961536 

961536


In [248]:
def trace_k(fhash, k , l):
    x_taille_graph = np.linspace(10,k,k-10,dtype=int)
    y_temps = np.zeros(k-10)
    tmp= np.zeros(5)
    for i in range(len(x_taille_graph)):
        
        for j in range(5):
            debut = time.time()
            collision_detection(fhash ,x_taille_graph[i] , l)
            fin = time.time()
            tmp[j] = fin-debut
        y_temps[i] = np.mean(tmp)

    plt.plot(x_taille_graph, y_temps, color= np.random.rand(3,), label=fhash.__name__)
    plt.gca().legend().set_visible(True)
    plt.title(fhash.__name__)
    plt.xlabel("k")
    plt.ylabel("temps de calcul (s)")
    plt.show()
    
def trace_l(fhash, k , l):
    x_taille_graph = np.linspace(2,l,l-2,dtype=int)
    y_temps = np.zeros(l-2)
    
    for i in range(len(x_taille_graph)):
        debut = time.time()
        collision_detection(fhash ,k , x_taille_graph[i])
        fin = time.time()
        y_temps[i] = fin-debut

    plt.plot(x_taille_graph, y_temps, color= np.random.rand(3,), label=fhash.__name__)
    plt.gca().legend().set_visible(True)
    plt.title(fhash.__name__)
    plt.xlabel("l")
    plt.ylabel("temps de calcul (s)")
    plt.show()
